In [14]:
from flask import Flask,render_template,url_for,request
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import heapq

In [15]:
app = Flask(__name__)

In [16]:
def html1(content):
   return '<html><head><title>Summarized</title></head><body><h1>Summarized Text</h1><hr><p>'+content+'</p></body></html>'

In [17]:
def html2(question, answer):
   return '<html><head><title>Summarized</title></head><body><h1>Answered Question</h1><hr><h4>Question: '+question+'</h4><p>Answer:'+answer+'</p></body></html>'

In [18]:
@app.route('/summarize',methods=['POST'])
def summarize():
    def summarizer(raw_text):
        stopWords = set(stopwords.words("english"))
        word_frequencies = {}  
        for word in nltk.word_tokenize(raw_text):  
            if word not in stopWords:
                if word not in word_frequencies.keys():
                    word_frequencies[word] = 1
                else:
                    word_frequencies[word] += 1

        maximum_frequncy = max(word_frequencies.values())

        for word in word_frequencies.keys():  
            word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

        sentence_list = nltk.sent_tokenize(raw_text)
        sentence_scores = {}  
        for sent in sentence_list:  
            for word in nltk.word_tokenize(sent.lower()):
                if word in word_frequencies.keys():
                    if len(sent.split(' ')) < 30:
                        if sent not in sentence_scores.keys():
                            sentence_scores[sent] = word_frequencies[word]
                        else:
                            sentence_scores[sent] += word_frequencies[word]



        summary_sentences = heapq.nlargest(10, sentence_scores, key=sentence_scores.get)

        summary = ' '.join(summary_sentences)  
        return summary


    if request.method == 'POST':
        documentText = request.form['documentText']
        
    return html1(summarizer(documentText))

In [19]:
@app.route('/answer',methods=['POST'])
def answer():
    import torch
    from transformers import BertTokenizer, BertForQuestionAnswering
    tokenizer = BertTokenizer.from_pretrained('./saved_tokenizing_model')
    model = BertForQuestionAnswering.from_pretrained('./saved_question_answering_model')
    #function that answers a question
    def answerQuestion(question,context):
        encodedText = tokenizer.encode("[CLS] " + question + " [SEP] " + context + " [SEP]")
        tokens = [0 if i <= encodedText.index(102) else 1 for i in range(len(encodedText))]
        startScore, endScore = model(torch.tensor([encodedText]), token_type_ids=torch.tensor([tokens]))
        allTokens = tokenizer.convert_ids_to_tokens(encodedText)  
        returnText = (' '.join(allTokens[torch.argmax(startScore) : torch.argmax(endScore)+1]))
        return str(returnText).replace("#", "")
        
    if request.method == 'POST':
        context = request.form['context']
        question = request.form['question']

    return html2(question,(answerQuestion(question, context)))

In [ ]:
if __name__ == '__main__':
    app.run(host="localhost", port=8003, use_reloader=False,debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://localhost:8003/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jun/2020 14:23:43] "POST /answer HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 14:23:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Jun/2020 14:47:23] "POST /summarize HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 14:47:40] "POST /summarize HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 14:48:34] "POST /answer HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2020 14:50:46] "POST /summarize HTTP/1.1" 200 -
